### Loading the corpus and tokenizing words

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading poetry for the corpus and adding start and end tags. The tokenized words are stored in the list total with start and end tags.


*   Creating a list for start words in list start_words




In [32]:
import copy
f = open('/content/drive/My Drive/NLP3_i18-0562/corpus.txt', 'r')

total = []   #Global list of tokenized words
start_words = []  # Global list of start words


Lines = f.readlines()

count = 0

temp = ''

for j in Lines:

  if len(j)<=3:
    continue

  words = j.split(' ')
  clean = []

  # Removing the \n from the last word
  for i in words:
    clean.append(i.strip())

  # Adding start words to the start words list,make sure that it is not repeated
  if clean[0] not in start_words:
    start_words.append(clean[0])


  #  Adding the start and end tags
  clean.insert(0,'<s>')
  clean.append('</s>')

  # print(clean)
  
  for i in clean:
    total.append(i) 





### Storing Rhyme Words

Making a rhyming dictionary

1. The makePairs() function takes in the list of tokenized words from corpus and return a list of pairs.For rhyming I will rhyme every two verses so the first step is to return a list of pair between every two consecutive verses.

2. The pairs returned from the makePairs() functions is sent to the makeRhymes() function. This function basically creates a dictionary where the key is an ending word and the value of that key contains a list of words that can be rhymed with that word.          Example  'نہیں': ['چلے', 'گیا', 'قضا']. Here the key is 'نہیں' and the list contains the words that can be rhymed in the next consecutive verse. In our generation function the rhyme word will be chosen randomly.

3. The fillEndList() function simply makes a list of end words filled with keys from the rhyme dictionary. A word will be chosen from this list for even verses in a stanza. For odd verses a word will be chosen randomly from the list of the chosen key. 



In [ ]:

def makePairs(t_list):
  pairs = []
  count = 1 
  for i in range(len(t_list)):
    if t_list[i]== '</s>':
      if count %2 == 1:
        temp1 = copy.deepcopy(t_list[i-1])
        count+=1
        # print("temp1->",temp1)
      elif count %2 == 0:
        temp2 = copy.deepcopy(t_list[i-1])
        # print("temp2->",temp2)
        if (temp1,temp2) not in pairs:
          pairs.append(tuple((temp1, temp2)))
        count+=1
  return pairs

pair = makePairs(total)
pair
      
def makeRhymes(pairs):
  rhymes = {}
  
  for i in pairs:
    temp = []
    if i[0] in rhymes:
      continue
    word = i[0]

    for j in pairs:     
      if j[0] == word:
        if j[1] not in temp:
          temp.append(j[1])  
    rhymes[i[0]] = temp
  return rhymes



# Storing all the keys in the end list function
def fillEndList(rhymes):
  e_list  = []
  for i in rhymes:
    e_list.append(i)

  return e_list


rhymes = makeRhymes(pair)


end_words = fillEndList(rhymes)  # Global list of end words,

end_words


### Unigram Model (with rhyming)

In [26]:
import spacy
import nltk
import random
import operator
from collections import defaultdict
from nltk.probability import ConditionalFreqDist
from nltk.util import ngrams
from collections import Counter
unlp = spacy.blank('ur')

Helper function which recives a list of verses and prints them in stanza formation.

In [27]:
def printVerses(v):

  c = 0  
  for j in v:
    c+=1
    for k in range(len(j)):
      print(j[k],end = " ")

    print() # end of each verse

    if(c %4==0):
      print() # end of each stanza

In [28]:
def countSingle(word,total):
  count = 0
  for i in total:
    if i == word:
      count+=1
  return count

def countDouble(word1,word2,total):
  count = 0
  for i in range(len(total)-1):
    if total[i] == word1 and total[i+1] == word2:
      count+=1
  return count

def countTriple(word1,word2,word3,total):
  count = 0
  for i in range(len(total)-2):
    if total[i] == word1 and total[i+1] == word2 and total[i+2] == word3:
      count+=1
  return count

In [29]:
def Unigram_Probabilities(total):

  u_dict = {}
  
  temp = []

  for i in total:
    if i in u_dict:
      continue
    count =  float(countSingle(i,total)/len(total))
    u_dict[i] = count

  sorted_dict = sorted(u_dict.items(), key=operator.itemgetter(1))

  # REMOVING START AND END TAGS
  del sorted_dict[-1]
  del sorted_dict[-1]

  return sorted_dict
  
  


Unigram poetry generator function which which generates poetry given tokenized list of words,start words,end words and rhyming list. No predictor function is required since no history required for each word.

Logic:
1. As seen above after sorting the dictionanry it is natural that the words witht the highest probabilty are going to be coneecting words such as سے ,کیا ,تو etc. These connecting words will be place at the end of the sorted list.

2. Similarly words with less probabilty but high value in poetry such as انقلاب' پیمانہ will have low probabilty and will be present in the beginnig of the sorted dictionary.

3. Thus for poetry with unigrams (no history) is to start with a word from the list of start words,then choose a coonecting word from the second half of the list followed by a high meaning word from the first half of the list and repeating this procedure until the end of the verse.

4. In this way meaning wise the verse will most likely be gibberish but its poosible to maintain some synctactic validity of each verse.





In [30]:
def Unigram_Poetry(t_words,s_words,e_words,r):

  print("     UNIGRAM MODEL POETRY\n")
  
  u_prob  = Unigram_Probabilities(total)

  verses = []
  
  e_word = ""

  for i in range(3):

    for j in range(4):

      temp = []  # List for storing all the words in this verse and will be appended to the list of lists (verses)

      # LENGTH OF VERSE
      length = random.randint(7,10)  
        
      #CHOOSING A START WORD AT RANDOM
      index =  random.randint(0,len(s_words)-1)
      start_word = s_words[index]
      
      temp.append(start_word)

      for k in range(length-2):

        # CHOOSE CONNECTING WORD
        if k%2 ==0:
          back = random.randint(1,int(len(u_prob)/2))
          temp.append(u_prob[-back][0])

        # CHOOSE HIGH MEANING WORD
        if k%2 ==1:
          front = random.randint(0,int(len(u_prob)/2))
          temp.append(u_prob[front][0])        

                # R H Y M I N G #
      # Choosing last word from list of end words (for even verses)
      if j%2 == 0:
        index2 =  random.randint(0,len(e_words)-1)
        e_word = e_words[index2]
        temp.append(e_words[index2])

      # Choosing rhyme word for rhyming with previous verse (for odd verses only)  
      elif j%2 == 1:
        rhyme_choice = r[e_word]
        choice_index = random.randint(0,len(rhyme_choice)-1)
        temp.append(rhyme_choice[choice_index])
      
      verses.append(temp)  # Appending the current entire verse to verses


  printVerses(verses)



Unigram_Poetry(total,start_words,end_words,rhymes)

     UNIGRAM MODEL POETRY

دیدہ دلستانی تازِ سوزن ہنِ کلام رودادِ گریباں لو‘ تھما 
بقدرِ تعمیر تا‘ بیٹهے خمیازۂ خلش آنسو‘ ہے؟ 
آتش لب سفینہ دروازہ رہیو جایا کے؟ چیز تعلیم 
سُراغِ ہوسِ بھید جولاں جُز آپکو کامل تنہائی بلبلِ تک 

مژدۂ غم پیراہن اس صرفِ آشیاں دبستاں دلستاں سکتا 
خانمانِ سرگشہٴ نِدا قد پھرا آساں‘ زبانوں اٹھتا ہوا 
انِ ملک ک طلبگارِ کشمیر پرواز سویداۓ ہوں! 
فلکِ چارہ نشینِ دنداں سودائے معاملہ پرفشاں ویرانی 

ہنوز قضا یاسِ نا امیدوار‘ بعد پارہ دی پر 
کمالِ پروانہ شاہنشاہ دعا خوبرویوں تک شکائتیں 
نگاہ لے خصم بازیِ نمط کو! خالِ ہوں! 
حق جذبہٴ ہا‘ ز جن ہمیں فروغ ویرانی 



###Simple Bigram Model  (with rhyming)

Bigram Probability Function

The first function is the most improtant function. This function calculates all  the bigram probabilitoes in a 2 Dimemesinal Dictionary.

1. Each key in the first dictionary is basically the first word.
2. Given key of first word the second dictionary is a list of words that can appear after the first word.Thei value is the probability of the second word appearing after the first word.
3. This function then return the 2D list which is used further by the predictor function of different model. 

EXAMPLE:  'آؤ': {'کریں': 0.5, 'کہ': 0.5}   is one of the entries in this dictionary.
1. Here 'آؤ' is the key of the first dictionary.
2. The value it holds is the second dictionary  {'کریں': 0.5, 'کہ': 0.5}.
3. One of the keys in the second dictionay is کریں and the value is 0.5.
4. This shows that the conditional probability of کریں appearing after آؤ is 0.5.

In [ ]:

def Bigram_Probabilty(total):

  mainDict = defaultdict(list)  # This dictionary holds a list of dictionaries

  previous_word = ""
  for i in total:
    if previous_word != "":
      mainDict[previous_word].append(i)
    previous_word = i

  for key in mainDict.keys():

    count = countSingle(key,total)
    # print(key,"->",count)
    next_words = mainDict[key]  # Getting the whole list of words after the key word
    unique_words = set(next_words)  # Remaove duplicates
    probabilities_key = {}

     # Now we will iterate over the list of words after the key word,and store probabilty with each word
    for u in unique_words:
      # print(key," ",u,"->",countDouble(key,u,total))
      count2 = countDouble(key,u,total)
      probabilities_key[u] = copy.deepcopy(count2/count)

    mainDict[key] = probabilities_key

  return mainDict
    

probs = copy.deepcopy(Bigram_Probabilty(total))
probs




The Bigram predictor Funtion is the second part of the bigram poetry generation that given all the probabilities and the first word how do we predict the next word.

Logic:
1. The first thing this function does is go to the value of the first dictionary given the first word.
2. As explained above the value of the fisrt dictionay key is another dictionary of next probabble word.
3. The second part is that the function iterates over the second dictionary and stores the maximum probabolity number only.
4. After we have the max number the function iterates again over this dictionary appending the key to the list choices if their value is equal to the max .
5. A list of all the choices is returned and the generation function can choose the next word randomly from all the list of choices.


In [10]:
def bigram_predictor(prob_list, word):

  max = -1
  choices = []
  if word in prob_list:
    # return prob_list[word]
    for i in prob_list[word]:
      if prob_list[word][i]> max and i != '</s>' and i != '<s>':
        max = prob_list[word][i]
    # print(max)

    for j in prob_list[word]:
      if prob_list[word][j] == max  and j != '</s>' and j != '<s>':
        choices.append(j)

  
  return choices


bigram_predictor(probs,"میں")


['نے']

Poetry generator function using bigrams

This function given takes in a list of tokenized list,start words, end words and dictionary of rhymes.

Logic:
1. Using the tokenized list a 2D dictionary of probabilties is recieved by calling the function Bigram_Probabilities().
2. For three stanzas each loop runs 3 times and and then the inner loop runs 4 times for each verse.
3. For each verse a random number is generated for its length. The first word is chosen from the list of start words randomly. After thes the next words till (lenght-1) are generated using the Bigram_Predictor function.
 RHYMING
4. The final word is chosen for a list of end words.The list of end words as explained in the STORING RHYME WORDS section contains words whose pair can be found in the rhyme dictionary.
5. Once the rhyme word is chosen for the even verses, in the subsequent odd verse the rhyme word is used as key in the rhyme dictionary which gives a list of possible rhyme word pair. The rhyme word is then chosen randomly.


In [11]:
def Simple_Bigram_Poetry(total,s_words,e_words,r):


  print("   SIMPLE BIGRAM MODEL POETRY  \n")

  # Storing all probabilities using bigram model
  verses = []
  probabilities = copy.deepcopy(Bigram_Probabilty(total)) # Getting all the probabilities

  e_word = ""

  for i in range(3):

    for j in range(4):

      temp = []  # List for storing all the words in this verse and will be appended to the list of lists verses

      length = random.randint(7,10)  # Length of the line
        
      #Choosing a start word at random for the start word list that was made when loading the corpus
      index =  random.randint(0,len(s_words)-1)
      start_word = s_words[index]
      
      temp.append(start_word)

      for k in range(length-2):

        options = bigram_predictor(probabilities,start_word) # Gettings the list of options words given the current word
        
        if(len(options)!= 0):
          
          if len(options)>1:
            c =  random.randint(0,len(options)-1)
            # print(options[c],end = " ")
            start_word = options[c]
            temp.append(start_word)

          elif options[0] != start_word:
            # print(options[0])
            start_word = options[0]
            temp.append(start_word)

          else:
            ind =  random.randint(0,len(s_words)-1)
            start_word = s_words[ind]
            temp.append(start_word)

        else:
          index1 =  random.randint(0,len(s_words)-1)
          start_word = s_words[index1]
          temp.append(start_word)
          # print(start_word,end = " ")

                # R H Y M I N G #
      # Choosing last word from list of end words (for even verses)
      if j%2 == 0:
        index2 =  random.randint(0,len(e_words)-1)
        e_word = e_words[index2]
        temp.append(e_words[index2])

      # Choosing rhyme word for rhyming with previous verse (for odd verses only)  
      elif j%2 == 1:
        rhyme_choice = r[e_word]
        choice_index = random.randint(0,len(rhyme_choice)-1)
        temp.append(rhyme_choice[choice_index])
      
      verses.append(temp)  # Appending the current entire verse to verses


  printVerses(verses)
  


Simple_Bigram_Poetry(total,start_words,end_words,rhymes)


   SIMPLE BIGRAM MODEL POETRY  

نہ ہو تو نے آخر شب میں غبار 
ہوائے قرطبہ شاید یہ بات کہ ہیں 
افکار سے گزر گیا تھا اے نگاہ تو لولاک 
بندہ ء ناموری ہے، نہ ہو تو نے چالاک 

کھویا گیا تھا اے نگاہ تو تشنگی 
اک نفس مرا تذکرہ جو ساقی آئیں 
زیتون کا یہ بات کہ اس کی طرح عیش تک 
خرید سکتے ہیں وہ خاک کہ دل اندھیرا 

نقش و نظر کا یہ بات آباد 
ندا آئی امتحاں اور بھی ہے یا میرا درد ہے 
زندگی کا ہر اک دن یہاں فرہاد 
غلط تھا اے رہرو کہ میں نے نے 



### Backward Bigram Model (with rhyming)

Backward Bigram Model (Without Rhyme Version)

Changes compared to the simple bigram model


1.   First of all the list of tokenized words is reversed.As a result the bigrams formed,their probabilty disctionanry and the bigram predictor function all do not need to be changed as they will work in the correct way predicting the next word in the same way.Only the word they will be predicting will actually be the previous word.
2.   The Second change is that the temp list will be reversed at the end before being appended to the verses(list of list containgi all verses).
3.   Third change is that instead of choosing a word from the start words the word will be chosen randomly from a list of end words.
4. The end word will be choses fro a list of start word





In [12]:
def Backward_Bigram_Poetry(main_list,s_words,e_words):


  print("   BACKWARD BIGRAM MODEL POETRY  \n")
  verses = []

  word_list = copy.deepcopy(main_list)
  word_list.reverse()

  # Storing all probabilities using bigram model
  probabilities = copy.deepcopy(Bigram_Probabilty(word_list))


  for i in range(3):

    for j in range(4):

      temp = []

      length = random.randint(7,10)  # Length of the line
        
      #Choosing a start word at random for the start word list that was made when loading the corpus
      index =  random.randint(0,len(e_words)-1)
      start_word = e_words[index]
      
      temp.append(start_word)

      for k in range(length-2):

        options = bigram_predictor(probabilities,start_word)
        
        if(len(options)!= 0):
          
          if len(options)>1:
            c =  random.randint(0,len(options)-1)         
            start_word = options[c]
            temp.append(start_word)

          elif options[0] != start_word:
            # print(options[0])
            start_word = options[0]
            temp.append(start_word)

          else:
            ind =  random.randint(0,len(e_words)-1)
            start_word = s_words[ind]
            temp.append(start_word)

        else:
          index1 =  random.randint(0,len(e_words)-1)
          start_word = e_words[index1]
          temp.append(start_word)

      # Placing the last words for a list of start words since this is backward model
      index2 =  random.randint(0,len(s_words)-1)
      temp.append(s_words[index2])

      temp.reverse()    # Reversing the list here to coorect the verse produced in reverse fashion
      verses.append(temp)
  
  #Printing the verses produced
  printVerses(verses)


Backward_Bigram_Poetry(total,start_words,end_words)

# back_bigram_verses

   BACKWARD BIGRAM MODEL POETRY  

تمہاری سے دل کا بندہ ء رفعت 
کار ہو جس سے دل میں نہ تھے 
افکار دل میں نہ ہو چکا ایسے نہیں ناصاف 
دیکھو مٹنا تجھے کیا ہے جس سے دل میں اچھے 

یہی کے رہنے والو خدا کے تلے 
آئے کافوری چاندنی کیا ہے یا شمع 
ڈھونڈ کی پرواز کیا ہے ، ایسے نہیں باقی 
دو آج کس خلش پیکاں آسودگئ فتراک 

ہوں پازند نقش کیا ہے طواف اولیٰ 
قلب جس سے دل میں نہ سمرقند 
ملا بہاری یہی کیا ہے اس کی آغوش 
ادھر صبح ازل کیا ہے کہ بڑھا کون کیا ہے 



Backward Bigram Model (Rhyme Version)

For Rhyme version the logic is same as the the without rhyme version. The difference is that

1. The start word is chosen from the list of end words for even verses.
2. For odd verses it is chosen using the the first word of previous verse.

(I am using the term first word here since it is the backward bigram model.In reality it is the last word.At the end the verse will be reversed and printed)

In [13]:
def R_Backward_Bigram_Poetry(main_list,s_words,e_words,r):

  print("   BACKWWARD BIGRAM MODEL POETRY  \n")

  verses = []

  word_list = copy.deepcopy(main_list)
  word_list.reverse()

  # Storing all probabilities using bigram model
  probabilities = copy.deepcopy(Bigram_Probabilty(word_list))


  for i in range(3):

    prev_word = ""

    for j in range(4):
   
      temp = []

      length = random.randint(7,10)  # Length of the line
             
      start_word = ""

                      # R H Y M I N G #
      # Choosing last word from list of first words (for even verses)
      if j%2 == 0:
        index =  random.randint(0,len(e_words)-1)
        start_word = e_words[index]
        temp.append(start_word)
        prev_word = copy.deepcopy(start_word)

      # Choosing rhyme word for rhyming with previous verse (for odd verses only)  
      elif j%2 == 1:
        rhyme_choice = r[prev_word]
        choice_index = random.randint(0,len(rhyme_choice)-1)
        temp.append(rhyme_choice[choice_index])


      for k in range(length-2):

        options = bigram_predictor(probabilities,start_word)
        
        if(len(options)!= 0):
          
          if len(options)>1:
            c =  random.randint(0,len(options)-1)         
            start_word = options[c]
            temp.append(start_word)

          elif options[0] != start_word:
            # print(options[0])
            start_word = options[0]
            temp.append(start_word)

          else:
            ind =  random.randint(0,len(s_words)-1)
            start_word = s_words[ind]
            temp.append(start_word)

        else:
          index1 =  random.randint(0,len(e_words)-1)
          start_word = e_words[index1]
          temp.append(start_word)

      # Placing the last words for a list of start words since this is backward model
      index2 =  random.randint(0,len(s_words)-1)
      temp.append(s_words[index2])

      temp.reverse()    # Reversing the list here to coorect the verse produced in reverse fashion

      verses.append(temp)
  
  #Printing the verses produced
  printVerses(verses)


R_back_bigram_verses = R_Backward_Bigram_Poetry(total,start_words,end_words,rhymes)


   BACKWWARD BIGRAM MODEL POETRY  

لب کیا ہے جس سے دل میں آ گیا 
یک نہ ہو جس سے ھیں کتنے اچھے گیا 
روز دل بہت کچھ اور بھی ہیں کئی دیپ 
عروج ناپختہ کیا ہے انبار خس و ہنر مند ھوں 

مدت خس و مئے ایّام، ورنہ فیض 
پرندے ہے بے رنگ و عشق و غرب تھے 
آیا سے دل میں نہ ہو تو کہاں 
دامن بند پابند کیا ہے یہ دیکھنا مقصود چراغ 

لاتے ہو تو کہاں کیا ہے رقیب 
ایمان ہے انبار خس و نظر ہوگا 
جب جس سے دل میں نہ ہو جس سے نومیدی 
غریب مجھ کو خدا کے آزاد ہے! 



### Bidirectional Bigram Model (with rhyming)

For this model the logic on which it will work is that the generated verse length will be divided into half. 
1. The first half will be generated using the the bigram probabilties of simple bigrams.
2. The second half will be generated using backward bigram probabilities.
3. Rhyming will be done in the backward bigram part such that the last word will be chosen from the dictionary of rhymes.

In [14]:
def Bidirectional_Bigram_Poetry(forward_list,s_words,e_words,r):

  print("   BIDIRECTIONAL BIGRAM MODEL POETRY  \n")

  verses = []

  backward_list = copy.deepcopy(forward_list)
  backward_list.reverse() # A reversed list for backward bigram model

  # 2D dictionanry to hold all forward bigram probs
  prob_forward = copy.deepcopy(Bigram_Probabilty(forward_list))

  # 2D dictionanry to hold all backward bigram probs
  prob_back = copy.deepcopy(Bigram_Probabilty(backward_list))

  #STANZA
  for i in range(3):  

    prev_word = ""

    #VERSE
    for j in range(4): 
   
      temp1 = []
      temp = []

      length = random.randint(7,10)  # Length of the line
             
      start_word = ""

      ################# SIMPLE BIGRAM PART  ####################

      #Choosing a start word at random for the start word list that was made when loading the corpus
      index =  random.randint(0,len(s_words)-1)
      start_word = s_words[index]
      
      temp1.append(start_word)

      for k in range(int((length-2)/2)):

        options1 = bigram_predictor(prob_forward,start_word) # Gettings the list of options words given the current word
        
        if(len(options1)!= 0):
          
          if len(options1)>1:
            c =  random.randint(0,len(options1)-1)
            # print(options1[c],end = " ")
            start_word = options1[c]
            temp1.append(start_word)

          elif options1[0] != start_word:
            # print(options1[0])
            start_word = options1[0]
            temp1.append(start_word)

          else:
            ind =  random.randint(0,len(s_words)-1)
            start_word = s_words[ind]
            temp1.append(start_word)

        else:
          index1 =  random.randint(0,len(s_words)-1)
          start_word = s_words[index1]
          temp1.append(start_word)
          # print(start_word,end = " ")

      # print("Forward-> ",temp1)
      

      ################# BACKWARD BIGRAM PART  ####################
      start_word = ""

                      # R H Y M I N G #
      # Choosing last word from list of first words (for even verses)
      if j%2 == 0:
        index =  random.randint(0,len(e_words)-1)
        start_word = e_words[index]
        temp.append(start_word)
        prev_word = copy.deepcopy(start_word)

      # Choosing rhyme word for rhyming with previous verse (for odd verses only)  
      elif j%2 == 1:
        rhyme_choice = r[prev_word]
        choice_index = random.randint(0,len(rhyme_choice)-1)
        temp.append(rhyme_choice[choice_index])


      for k in range(int((length-2)/2)):

        options = bigram_predictor(prob_back,start_word)
        
        if(len(options)!= 0):
          
          if len(options)>1:
            c =  random.randint(0,len(options)-1)         
            start_word = options[c]
            temp.append(start_word)

          elif options[0] != start_word:
            # print(options[0])
            start_word = options[0]
            temp.append(start_word)

          else:
            ind =  random.randint(0,len(s_words)-1)
            start_word = s_words[ind]
            temp.append(start_word)

        else:
          index1 =  random.randint(0,len(e_words)-1)
          start_word = e_words[index1]
          temp.append(start_word)

      # Placing the last words for a list of start words since this is backward model
      index2 =  random.randint(0,len(e_words)-1)
      temp.append(e_words[index2])

      temp.reverse()    # Reversing the list here to coorect the verse produced in reverse fashion
      # print("Backward-> ",temp)

      verse = temp1 + temp
      verses.append(verse)  # Appending the current entire verse to verses

  printVerses(verses)




Bidirectional_Bigram_Poetry(total,start_words,end_words,rhymes)

   BIDIRECTIONAL BIGRAM MODEL POETRY  

نفس مرا شعلہ بار بنیاد ہے اس کی بہار 
بندہ بنوں گا سارا نظر دل و زندیق گزر 
حاضر ہے یا میرا شعیہ ٹھکانہ ابھی وہی طہٰ 
گو اس کی طرح بغداد ہے یہ توفیق ورنہ 

ستم کش انتظار ہوگا افلاک ہو تو نہیں ھیں 
مگر یہ بات خلاف نہ سمرقند اشارے 
بندہ بنوں گا جس جائیو میں نہ ہو طلب 
سبق شاہیں بناتا نہیں ہے دریا پھر کسے خبر کرو آرہوں 

آؤ نہ ہو تو ہو ہو تو خود را 
لوگوں کو بھی ہے یا کتنی بھی ہیں روز حساب نے 
جگنو کی طرح عیش سبق گزر ایسے نہیں بدلی 
لو وصل کی طرح پیہم و مستی گفتار کا 



### Trigram Model (with rhyming)

Trigram Probability calculation function

This function is similar in logic to the bigram probabilty function however the key of the key of the first dictionary in the 2D dictionary is a tuple of two words rather then just being a single word.

EXAMPLE:

('کبھی', 'تو'): {'شب': 0.5, 'صبح': 0.5'}

1. Here ('کبھی', 'تو') is the key of the first dictionary.
2. 'صبح' and 'شب' are keys in the second dictionary.
3. The probabilty of 'شب' occuring after ('کبھی', 'تو') is 0.5.

In [ ]:
def Trigram_Probabilty(total):

  mainDict = defaultdict(list)  # This dictionary holds a list of dictionaries

  back2 = ""
  back1 = ""
  for i in range(len(total)):
    if i > 0:
      if back1 != "" and back2 != "":
        mainDict[(back2,back1)].append(total[i])
      back1 = total[i]
      back2 = total[i-1]

  for key in mainDict.keys():

    count = countDouble(key[0],key[1],total)
    # print(key,"->",count)
    next_words = mainDict[key]  # Getting the whole list of words after the key word
    unique_words = set(next_words)  # Remaove duplicates
    probabilities_key = {}

     # Now we will iterate over the list of words after the key word,and store probabilty with each word
    for u in unique_words:
      # print(key," ",u,"->",countDouble(key,u,total))
      count2 = countTriple(key[0],key[1],u,total)
      probabilities_key[u] = copy.deepcopy(count2/count)

    mainDict[key] = probabilities_key

  return mainDict


tri_prob = Trigram_Probabilty(total)
tri_prob



Trigram Predictor Function

1. The logic is again similar to the bigram predictor function.

2. The Difference is that the function recieves two words instead of one since this is a trigram model and two words are needed to access the key of the first dictioanry.

3. Finally it returns a list of choices given the previous two words and the next word is chosen randomly for the choices by the generation function

In [16]:
def trigram_predictor(prob_list, word1,word2):

  max = -1
  choices = []
  if (word1,word2) in prob_list:
    # return prob_list[word]
    for i in prob_list[(word1,word2)]:
      if prob_list[(word1,word2)][i]> max and i != '</s>' and i != '<s>':
        max = prob_list[(word1,word2)][i]
    # print(max)

    for j in prob_list[(word1,word2)]:
      if prob_list[(word1,word2)][j] == max  and j != '</s>' and j != '<s>':
        choices.append(j)

  
  return choices


trigram_predictor(tri_prob,'آ', 'کے')

['دوبارا', 'چمکا', 'چلا']

Trigram poetry generator

Logic:

1. The first word is choses randomly from the list of start words,similar to the bigram model.
2. This function also calculates tht bigram probabilites and using the bigram predictor function the second word of each verse is printed using the bigram predictor function for accuracy  and so that the second word is not disjointed in meaning or style.
3. After having generated the first two words the remaing words of the verses are generated using the trigram predictor function.
RHYMING
4. The final word is is chosen from a list of end words for even verses.
5. For odd verses the rhyme option is chosen randomly given the end word of the previous verse as the key. 

In [17]:
def Trigram_Poetry(t_words,s_words,e_words,r):

  print("   TRIGRAM MODEL POETRY  \n")

  verses = []

   # Getting all the probabilities for trigram
  trigram_probs = copy.deepcopy(Trigram_Probabilty(t_words))

  # Getting all probabilties from bigram model to predict the second word in each verse
  bigram_probs = copy.deepcopy(Bigram_Probabilty(t_words))  

  e_word = ""

  for i in range(3):

    for j in range(4):

      temp = []  # List for storing all the words in this verse and will be appended to the list of lists verses

      start_word = ""
      start_word2 = ""

      length = random.randint(7,10)  # Length of the line
        
      #Choosing a start word at random for the start word list that was made when loading the corpus
      index =  random.randint(0,len(s_words)-1)
      start_word = s_words[index]
      
      temp.append(start_word)

      # Predicting second word using bigram probabilties for more accuracy
      ##################################
      b_options = bigram_predictor(bigram_probs,start_word)
      if(len(b_options)!= 0):
          
        if len(b_options)>1:
          c =  random.randint(0,len(b_options)-1)
          # print(options[c],end = " ")
          start_word2 = b_options[c]
          temp.append(start_word2)

        elif len(b_options) == 1 and b_options[0] != start_word:
          # print(options[0])
          start_word2 = b_options[0]
          temp.append(start_word2)

        else:
          ind =  random.randint(0,len(s_words)-1)
          start_word2 = s_words[ind]
          temp.append(start_word2)

      else:
        index1 =  random.randint(0,len(s_words)-1)
        start_word2 = s_words[index1]
        temp.append(start_word2)


      ############################################


      # USING START_WORD AND START_WORD2 FOR TRIGRAM CALCULATIONS

      # ******* #
      for k in range(length-3):

        start_word3 = ""
        options = trigram_predictor(trigram_probs,start_word,start_word2) # Gettings the list of options words given the current word
        
        if(len(options)!= 0):
          
          if len(options)>1:
            c =  random.randint(0,len(options)-1)
            # print(options[c],end = " ")
            start_word3 = options[c]
            temp.append(start_word3)

          elif len(options) == 1 and options[0] != start_word2:
            # print(options[0])
            start_word3 = options[0]
            temp.append(start_word3)

          else:
            ind =  random.randint(0,len(s_words)-1)
            start_word3 = s_words[ind]
            temp.append(start_word3)

        else:
          index1 =  random.randint(0,len(s_words)-1)
          start_word3 = s_words[index1]
          temp.append(start_word3)

        start_word = copy.deepcopy(start_word2)
        start_word2 = copy.deepcopy(start_word3)
          

                # R H Y M I N G #
      # Choosing last word from list of end words (for even verses)
      if j%2 == 0:
        index2 =  random.randint(0,len(e_words)-1)
        e_word = e_words[index2]
        temp.append(e_words[index2])

      # Choosing rhyme word for rhyming with previous verse (for odd verses only)  
      elif j%2 == 1:
        rhyme_choice = r[e_word]
        choice_index = random.randint(0,len(rhyme_choice)-1)
        temp.append(rhyme_choice[choice_index])
      
      verses.append(temp)  # Appending the current entire verse to verses


  printVerses(verses)
  

Trigram_Poetry(total,start_words,end_words,rhymes)

   TRIGRAM MODEL POETRY  

سنا ہے یہ قدسیوں سے میں نے وہ شیر میری 
چاند سے ماند ستاروں نے کہا آخر شب ہے! 
بُھولے تو یوں کہ جیسے ہمیشہ آئے 
افلاک پر بھول جدا ایک بہت میرے لبریز آئے 

خدا کے بندوں سے پیار ہوگا لطیفۂ ناداں خوف مہرباں 
دیکھو تو کدھر آج رخ باد صبا آخر تھے 
پہنچوں کس طرح غوروفکر سے بےنام قوم الا 
لمس جانانہ لیے مستیٔ پیمانہ لیے کریں بغل نے 

سونپا تھا جسے کام نگہبانئ دل کا خاشاک 
علم چارہ گر کو چارہ گری سے آنسو 
میں نے وہ شیر پھر ہوشیار منزل 
چپ رہ نہ سکا قند اگر فقط علاج ندا آئے 



### Comparisons Between Models

UNIGRAM MODEL

Analysis:
As it can be seen the unigram model even with the logic of choosing one connecting word and one high value word does not make much sense compred to the other model. Every two words in each verse are disjointed and do not feel right synctactically.

In [18]:
Unigram_Poetry(total,start_words,end_words,rhymes)

     UNIGRAM MODEL POETRY

بے قلندری طریقت بندے لمس سخن گوارا خواروں ید آیا 
بندگی وجدان بھروسا دھمکانے اڑا گھڑی محشر پر 
دماغ پرویز قرطبہ حکم کاروبار اچھے زنجیر شے 
کتابِ لاکھ مغربیاں شکوے برپا دیکھو فرہاد پرویز دماغ 

وہ ذکر مثل خاشاک کافری گستاخ ناصاف شعیہ 
ترے آنکھوں جنگاہ جا کاربندِ یارو شاہین سکندری واسطہ 
رکھتی افق طریقت فلک شدہ شیشہ بتاں ، پسند جسے 
گنو شاعر دہناں تمدن ٹوٹا حکم قلم چگنے دیا 

بدلتا فرزندی جوئے جاتی سولی فاش آئے 
بڑی آزما اندیش انداز اٹھتی حجاب آئے 
سلوک والو اویس جب عجیب گزار چلو ہجراں 
کسی بتاؤں گلے  خلیق دور چلے 



SIMPLE BIGRAM MODEL

Analysis:
The backward bigram model is a huge improvement over the unigram model. The verses for the most part fulfill the synctactic criteria but meaning wise they feel a little disjointed after every 2-3 words in each verse. The rhyming as it can be seen fits well but it does not always join smoothly with the words behinf it. This is bacause the last word is chosen as soon as the loop lenght of the verse runs out. It is not chosen using the bigram prediction.

In [19]:
Simple_Bigram_Poetry(total,start_words,end_words,rhymes)

   SIMPLE BIGRAM MODEL POETRY  

حمد باری کو بھی ہے یا سنائیو 
تری زیبائی سے گزر گیا ہے یا کریں 
افکار کی طرح عیش نیام سے گزر گیا تھا کی 
زمانے ميں تفاوت نہيں ليکن ذرا سا چھپا تھے 

صنم آشنا تجھے مثال شرار ہوگا نو نومبر چاھے 
اس کی طرح عیش نیام سے گزر گیا تھا ھے 
بھروسا کر کوئی بات کہ دل اترے 
جہان مے خانہ سن کے دیکھ لیا ھو آئے 

ہزار کر کے دیکھ لیا جسے حق نے سخن آغوش 
اندیشۂ چالاک طرب آشنائے خروش ہو تو نے کیا نہیں 
رہے ہیں وہ خاک کہ میں نے بادہ خوار استغنا 
ہنسی آتی ہے یا میرا دل کا کی 



BACKWARD BIGRAM MODEL

Analysis:
The Backward Bigram model is similar to the simple bigram model in this regard.The last word feels better here since the words generated before it are predicted using the bigram predictor function. However now the problem occurs slightly as the forst word feels a little disjointed since it is chosen after the loop to end the verse. Syntax wise the verses are correct for the most part however meaning wise it is still poor.

In [20]:
R_Backward_Bigram_Poetry(total,start_words,end_words,rhymes)

   BACKWWARD BIGRAM MODEL POETRY  

مہ دل میں کچھ اور بھی ہیں زیاد 
وضع مجھ کو بخشی سختی خارا حجاب 
عروج ہو تو جاں کیا غم فرہاد 
کون سے دل میں نہ سمجھو نے 

دبا سے گزر کیا ہے یہ کہکشاں برگیبانِ مرغزار 
جدا دل کا سفر نظارۂ بام سحاب 
بنگر باغ بہشت مغربیاں جلوہ ہائے پا برکاب 
متاع دل میں نہ ہو تو جاں عدو جا 

نئی دل میں نہ ہو تو پھر کسے خبر کرو 
نگہ نہ ہو تو تھیں قربتیں کتنی دیا 
بنااُڑے مجھ کو خدا کے رہنے والو خدا کے لانا 
ترے جس سے دل میں نہ سمجھو یارو 



BIDIRECTIONAL BIGRAM MODEL

Analysis:The bigram model is the best of both simple bigram and backward bigram model since there is no disjoint meaning at the start and end of most of the verses as seen below. However the logic used is that half of the verse is generated from the back and half from the front. As a result sometimes the verse is slightly dijointed at the middle of the verse.
Meaning wise the model is similar to backward bigram and simple bigram model. 

In [21]:
Bidirectional_Bigram_Poetry(total,start_words,end_words,rhymes)

   BIDIRECTIONAL BIGRAM MODEL POETRY  

حمد باری کو میری تھیں قربتیں کتنی 
سچ ہے یا میرا دیا میں اس کی کیا 
تر دامنی پہ یارو پرویز میں نہ ہو تو 
چیتے کا ہر اک صیاد باقی ھو تشنگی تھا 

عقل کو نہ ہو سمجھا جس سے دل غریب 
مقامات آہ و نظر لاتخف خدا کے در اپنا 
چاند کے دیکھ دریا زمانے بھول گئے 
لرزیدہ میری نفس پر خبر کرو تھا 

سنائے کون غزل خواں ہے پروا میں نہ کر کوئی ہدف 
مے خانہ ہر اک نفس اعراف نہ کر کوئی ستارہ نہیں 
قافلہِ عشق و نظر پازند جو شاخ یقیں نمناک 
رہ جا تو نے ہمیں اعراف ہے جس سے خموش بیباکی 



TRIGRAM MODEL

Analysis: The trigram model is by far the best of all the model as seen from the poetry generated below. Syntax wise there is almost no errors or disjoint parts. This model is even superior meaning wise as some verses actually qualify for poetry to some extent. The model chooses the forst word at random but the second word is chosen using bigram and the subsequents words using trigrams. As a relsult the syntax is so smooth that even the last word which is chosen wothout any context seems to fit in most verses.

In [33]:
Trigram_Poetry(total,start_words,end_words,rhymes)

   TRIGRAM MODEL POETRY  

دامن نچوڑ دیں تو فکر دل و نظر کا گئے 
کتابِ خودی کا انوکھا کردار ہے تھا 
کبھی تو صبح ترے کنج لب سے ہو بیتابی 
شمع نے یہ کہا کہ وہ اثر کہن اگرچہ آیا 

تم سمجھ رہے ہو وہ اب زر یعنی 
اسے اب چاند کے غاروں میں نظر بند پایا 
گلیوں میں پھرا کرتے تھے دو چار دوانے جہان گداز 
نام پہ خالی سبو کریں پرسوز ڈھونڈھنا 

تھک کر یوں ہی پل بھر نزول 
تاویل سے قرآں کو بنا سکتے ہیں کھلا 
قناعت نہ کر تلف چمکا صوفی ضبط سینے 'کیا'؟ 
قصوروار غریب الدیار ہوں لیکن نکل راہ 

